In [1]:
import random
import numpy as np
import pandas as pd
import time
import re
import json
import datetime
import json
import os
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import pickle

In [36]:
# pip install sklearn

In [16]:
# pip install scikit-network

### Read networks

In [ ]:
import sknetwork
from sknetwork.clustering import Louvain

In [46]:
dataDict = dict()
for cat in ['western','russian']:
    with open(f'visualizations/ngramsNetwork/{cat}ngramNet5_8-5_w_nodeandlinkpolarity.json','r') as json_file:
        dataDict[cat] = json.load(json_file)

In [47]:
for cat in dataDict:
    print(len(dataDict[cat]['nodes']),len(dataDict[cat]['links']))

1032 41867
1446 75849


In [61]:
len([n for n in dataDict['western']['nodes'] if n['occurrences'] >= 600])

16

In [63]:
len([n for n in dataDict['russian']['nodes'] if n['occurrences'] >= 750])

30

In [65]:
dataDict['russian']['links'][10]

{'source': 1138, 'target': 184, 'value': 1, 'polarityScore': -0.318}

In [68]:
smallJson = {'nodes' : [], 'links': []}
j = 0
newNodes = set()
old2new = dict()
for i, n in enumerate(dataDict['russian']['nodes']):
    if n['occurrences'] >= 750:
        old2new[i] = j
        smallJson['nodes'].append(n)
        newNodes.add(i)
        j += 1
for l in dataDict['russian']['links']:
    s = l['source']
    t = l['target']
    if s in newNodes and t in newNodes:
        newLink = l.copy()
        newLink['source'] = old2new[s]
        newLink['target'] = old2new[t]
        smallJson['links'].append(newLink)

In [78]:
numNodes = len(smallJson['nodes'])
adjArray = np.zeros((numNodes, numNodes), dtype=int)
for l in smallJson['links']:
    i = l['source']
    j = l['target']
    adjArray[i,j] = l['value']
    adjArray[j,i] = l['value']

In [81]:
louvain = Louvain()
labels = louvain.fit_transform(adjArray)
len(set(labels))

4

In [82]:
louvain.labels_

array([0, 0, 0, 0, 2, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 3, 1, 3,
       0, 0, 2, 0, 1, 0, 1, 1])

In [86]:
for i, n in enumerate(smallJson['nodes']):
    n['group'] = int(louvain.labels_[i])

In [87]:
with open(f'visualizations/adjmatrix/sample_russian.json', 'w', encoding ='ascii') as json_file:
    json.dump(smallJson, json_file, ensure_ascii = True)
print('wrote json')

wrote json


## Topic-specific matrix

In [93]:
import itertools
import stanza

In [88]:
%%time
sdf = pd.read_excel('data/topic_model_data/fullSentDataFrame_8-3.xlsx',index_col=0)
sdf = sdf.drop_duplicates(['sentText'])
sdf.head()

CPU times: user 14.9 s, sys: 1.9 s, total: 16.8 s
Wall time: 23.9 s


,index,newsOutlet,category,dateSeen,url,title,language,sourceCountry,sentText,sentIndexInText,sentTopicID
443,21,apnews.com,western,2021-10-01 07:00:00,https://apnews.com/article/2020-tokyo-olympics...,Poland grants visa to Belarus Olympian who fea...,English,United States,TOKYO (AP) — Poland granted a visa Monday to a...,0,-1
445,21,apnews.com,western,2021-10-01 07:00:00,https://apnews.com/article/2020-tokyo-olympics...,Poland grants visa to Belarus Olympian who fea...,English,United States,The current standoff apparently began after Ts...,2,-1
446,21,apnews.com,western,2021-10-01 07:00:00,https://apnews.com/article/2020-tokyo-olympics...,Poland grants visa to Belarus Olympian who fea...,English,United States,The runner said on her Instagram account that ...,3,-1
447,21,apnews.com,western,2021-10-01 07:00:00,https://apnews.com/article/2020-tokyo-olympics...,Poland grants visa to Belarus Olympian who fea...,English,United States,The runner was then apparently hustled to the ...,4,-1
448,21,apnews.com,western,2021-10-01 07:00:00,https://apnews.com/article/2020-tokyo-olympics...,Poland grants visa to Belarus Olympian who fea...,English,United States,In a filmed message distributed on social medi...,5,-1


In [94]:
stanza.download('en')
nlp = stanza.Pipeline('en')

2022-08-08 10:02:56 INFO: Downloading default packages for language: en (English)...
2022-08-08 10:02:59 INFO: File exists: /Users/paigelee/stanza_resources/en/default.zip.
2022-08-08 10:03:05 INFO: Finished downloading models and saved to /Users/paigelee/stanza_resources.
2022-08-08 10:03:05 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2022-08-08 10:03:05 INFO: Use device: cpu
2022-08-08 10:03:05 INFO: Loading: tokenize
2022-08-08 10:03:05 INFO: Loading: pos
2022-08-08 10:03:06 INFO: Loading: lemma
2022-08-08 10:03:06 INFO: Loading: depparse
2022-08-08 10:03:07 INFO: Loading: sentiment
2022-08-08 10:03:08 INFO: Loading: ner
2022-08-08 10:03:09 INFO: Done loading processors!


In [91]:
import nltk

from nltk import pos_tag, word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentiText
nltk.download('vader_lexicon')
from nltk.corpus import stopwords

engStopwords = set(stopwords.words('english'))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/paigelee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/paigelee/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### Pre-compute lemmas and polarity

In [116]:
allSentRecords = sdf.to_dict('records')

for i, rec in enumerate(allSentRecords):
    if i % 1000 == 0:
        print(f'{i}/{len(allSentRecords)} recs parsed by NLP')
    docText = rec['sentText']
    nlpdoc = nlp(docText)

    ss = sid.polarity_scores(docText)
    polarityScore = ss['pos'] - ss['neg']

    wordList = [w.lemma.lower() for w in nlpdoc.iter_words() if w.text.isalpha() and w.text.lower() not in engStopwords]
    rec['polarityScore'] = polarityScore
    rec['wordList'] = wordList

0/53349 recs parsed by NLP
1000/53349 recs parsed by NLP
2000/53349 recs parsed by NLP
3000/53349 recs parsed by NLP
4000/53349 recs parsed by NLP
5000/53349 recs parsed by NLP
6000/53349 recs parsed by NLP
7000/53349 recs parsed by NLP
8000/53349 recs parsed by NLP
9000/53349 recs parsed by NLP
10000/53349 recs parsed by NLP
11000/53349 recs parsed by NLP
12000/53349 recs parsed by NLP
13000/53349 recs parsed by NLP
14000/53349 recs parsed by NLP
15000/53349 recs parsed by NLP
16000/53349 recs parsed by NLP
17000/53349 recs parsed by NLP
18000/53349 recs parsed by NLP
19000/53349 recs parsed by NLP
20000/53349 recs parsed by NLP
21000/53349 recs parsed by NLP
22000/53349 recs parsed by NLP
23000/53349 recs parsed by NLP
24000/53349 recs parsed by NLP
25000/53349 recs parsed by NLP
26000/53349 recs parsed by NLP
27000/53349 recs parsed by NLP
28000/53349 recs parsed by NLP
29000/53349 recs parsed by NLP
30000/53349 recs parsed by NLP
31000/53349 recs parsed by NLP
32000/53349 recs pars

In [118]:
# with open('Sentence_Data_with_Polarity_and_Lemma_Lists.pickle', 'wb') as handle:
#     pickle.dump(allSentRecords, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
with open('Sentence_Data_with_Polarity_and_Lemma_Lists.pickle', 'rb') as handle:
    sentenceRecs = pickle.load(handle)

In [3]:
random.choice(sentenceRecs)

{'index': 250,
 'newsOutlet': 'apnews.com',
 'category': 'western',
 'dateSeen': datetime.datetime(2022, 1, 26, 8, 0),
 'url': 'https://apnews.com/article/russia-ukraine-russia-malaysia-europe-moscow-ff98b54807f2f6c74902504f3ba0db03',
 'title': 'Netherlands , Ukraine take Russia to European rights court',
 'language': 'English',
 'sourceCountry': 'United States',
 'sentText': 'In a statement, Dutch Foreign Minister Wopke Hoekstra called Wednesday’s hearing “an important step in the Netherlands’ pursuit of justice for the victims and their next of kin.',
 'sentIndexInText': 16,
 'sentTopicID': 14,
 'polarityScore': 0.09200000000000001,
 'wordList': ['statement',
  'dutch',
  'foreign',
  'minister',
  'wopke',
  'hoekstra',
  'call',
  'wednesday',
  'hearing',
  'important',
  'step',
  'netherlands',
  'pursuit',
  'justice',
  'victim',
  'next',
  'kin']}

## Run co-occurrence code

In [4]:
import itertools

In [16]:
# %%time
# getallSentRecordsccurrence connections
jsonDict = dict()

thresh = 100
n = 5

totalWordCt = {
    'western' : 0,
    'russian' : 0
}
topicDict = dict()
for topicID in range(-1,15):
    topicDict[topicID] = dict()
    for cat in ['western','russian']:
        networkJson = dict()
        networkJson['nodes'] = []
        networkJson['links'] = []

        curatedNodes = set()
        linkCounter = dict()
        keywordCounter = dict()
        polarityOfNode = dict()

        windowList = []
        records = []
        for d in sentenceRecs:
            if d['category'] == cat and d['sentTopicID'] != -1 and d['sentTopicID'] == topicID:
                records.append(d)

        for i, rec in enumerate(records):
            if i % 1000 == 0:
                print(f'{i}/{len(records)} recs parsed by NLP')

            polarityScore = rec['polarityScore']
            wordList = rec['wordList']

            for windowIdx in range(len(wordList) // n + len(wordList) % n):
                startIdx = windowIdx
                endIdx = windowIdx+random.choice(list(range(1,n+1)))
                wordWindow = wordList[startIdx:endIdx]
                tokens = []
                for wordToken in wordWindow:
                    formattedToken = wordToken
                    tokens.append(formattedToken)
                windowList.append((tokens, polarityScore))
            for t1 in wordList:
                keywordCounter.setdefault(t1, 0)
                keywordCounter[t1] += 1
                polarityOfNode.setdefault(t1, [])
                polarityOfNode[t1].append(polarityScore)
                totalWordCt[cat] += 1
            if totalWordCt[cat] >= 150000:
                print(f'stopping the {cat} count at {totalWordCt[cat]}words')
                break
        print('\tdone nlping docs')
        sufficientNodes = set()
        for key in keywordCounter:
            if keywordCounter[key] >= thresh:
                sufficientNodes.add(key)
        print('\tfound sufficient recs')
        polarityOfLinks= dict()

        for tokens, polarityScore in windowList:
            s = set(tokens)
            subsets = [tuple(i) for i in itertools.combinations(s, 2)]
            for t1, t2 in subsets:
                if t1 != t2 and t1 in sufficientNodes and t2 in sufficientNodes:
                    label = '_'.join(sorted([t1, t2]))
                    linkCounter.setdefault(label, 0)
                    linkCounter[label] += 1
                    polarityOfLinks.setdefault(label, [])
                    polarityOfLinks[label].append(polarityScore)
        print('\tgot linked tokens')
        node2id = dict()
        for i, node in enumerate(sufficientNodes):
            networkJson['nodes'].append({
                'id' : node,
                'occurrences' : keywordCounter[node],
                'nodePolarity' : sum(polarityOfNode[node])/len(polarityOfNode[node])
            })
            node2id[node] = i

        for label in linkCounter:
            source, target = label.split('_')
            networkJson['links'].append({
                'source' : node2id[source],
                'target' : node2id[target],
                'value' : linkCounter[label],
                'polarityScore' : sum(polarityOfLinks[label])/len(polarityOfLinks[label])
            })

        jsonDict[cat] = networkJson
        topicDict[topicID][cat] = networkJson
        print(f'successfully parsed {len(networkJson["nodes"])} nodes and {len(networkJson["links"])} links')

        # with open(f'visualizations/adjmatrix/topic_data/{cat}network{n}_topic{topicID}_adjmatrix.json', 'w', encoding ='ascii') as json_file:
        #     json.dump(networkJson, json_file, ensure_ascii = True)
        # print('\twrote json for topic',topicID)

	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 0 nodes and 0 links
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 0 nodes and 0 links
0/835 recs parsed by NLP
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 8 nodes and 24 links
0/796 recs parsed by NLP
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 9 nodes and 34 links
0/267 recs parsed by NLP
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 2 nodes and 1 links
0/410 recs parsed by NLP
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 3 nodes and 3 links
0/470 recs parsed by NLP
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 1 nodes and 0 links
0/622 recs parsed by NLP
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 1 nodes and 0 links
0/114 recs parsed by NLP
	done nlping docs
	foun

## T-test on sentiment

## Louvain clustering

In [108]:
for cat in ['western','russian']:
    smallJson = jsonDict[cat]

    numNodes = len(smallJson['nodes'])
    adjArray = np.zeros((numNodes, numNodes), dtype=int)
    for l in smallJson['links']:
        i = l['source']
        j = l['target']
        adjArray[i,j] = l['value']
        adjArray[j,i] = l['value']

    louvain = Louvain()
    labels = louvain.fit_transform(adjArray)
    len(set(labels))

    for i, n in enumerate(smallJson['nodes']):
        n['group'] = int(louvain.labels_[i])

    n = 5
    with open(f'visualizations/adjmatrix/{cat}network{n}_topic{topicID}_adjmatrix.json', 'w', encoding ='ascii') as json_file:
        json.dump(smallJson, json_file, ensure_ascii = True)